In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect
import pandas as pd

In [4]:
# create engine to hawaii.sqlite
hawaii_database_path ="Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{hawaii_database_path}")
conn = engine.connect()

In [5]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [6]:
# Collect the names of tables within the database
inspector.get_table_names()

['measurement', 'station']

In [7]:
# For measurement table
# Get a list of column names and types 
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [8]:
# For station table
# Get a list of column names and types
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [9]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [10]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['measurement', 'station']

In [11]:
# Assign the classes to two variables called `Measurement` and 'Station' 
measurement = Base.classes.measurement
station = Base.classes.station

In [12]:
# Create a session
session = Session(engine)

In [19]:
#For measurement class
# Display the row's columns and data in dictionary format 
first_row = session.query(measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2905f675c70>,
 'id': 1,
 'date': '2010-01-01',
 'tobs': 65.0,
 'prcp': 0.08,
 'station': 'USC00519397'}

In [21]:
# first_row = session.query(measurement.date,measurement.prcp).all()
first_row = session.query(measurement).all()
first_row

 ...]

In [22]:
all_names = list(np.ravel(first_row))
all_names

 ...]

In [ ]:
# Query All Records in the the measurement Table
df_measurement = pd.read_sql("SELECT * FROM measurement", conn)
df_measurement

In [ ]:
#For Station class
# Display the row's columns and data in dictionary format
first_row = session.query(station).first()
first_row.__dict__

In [ ]:
# Query All Records in the the Station Table
df_station = pd.read_sql("SELECT * FROM station", conn)
df_station

# Exploratory Precipitation Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data 
# and plot the results. 
# Starting from the most recent data point in the database. 

In [ ]:
# Measurement table
# Total dates
session.query(func.count(measurement.date)).all()

In [ ]:
# Find the most recent date in the data set.
session.query(measurement.date).order_by(measurement.date.desc()).first()

In [27]:
# Query for the measurement date 12 months before `2017-08-23` 
# using the datetime library
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)

Query Date:  2016-08-23


In [ ]:
# Query for the date and precipitation (open, high, low, close) 
# from the last 12 months (2016-8-23 to 2017-8-23)
# Sort the result by date

date_prcp = session.query(measurement.date, measurement.prcp).\
    filter(measurement.date >= '2016-08-23',measurement.prcp >= 0).all()
date_prcp

In [ ]:
# Plot the Results in a Matplotlib bar chart
df_date = pd.DataFrame(date_prcp, columns=['date', 'Precipitation'])
df_date.set_index('date', inplace=False)
df_date.plot(x="date", y="Precipitation", rot=90)
plt.xlabel("Date")
plt.ylabel("Inches")
plt.legend(loc='upper right')
plt.figure(figsize=(20,20))
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df_measurement['prcp'].describe()

# Exploratory Station Analysis

In [23]:
# Design a query to calculate the total number stations in the dataset
# Measurement table
# Total dates
session.query(func.count(station.station)).all()

[(9,)]

In [24]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

session.query(station.station).order_by(func.count(station.station).desc()).group_by(station.station).all()

[('USC00519523',),
 ('USC00519397',),
 ('USC00519281',),
 ('USC00518838',),
 ('USC00517948',),
 ('USC00516128',),
 ('USC00514830',),
 ('USC00513117',),
 ('USC00511918',)]

In [26]:
session.query(station.station).order_by(func.count(station.station).desc()).group_by(station.station).first()

('USC00519523',)

In [ ]:
df_measurement.columns

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

# List all of the measurment records for the most active station id
stn1 = session.query(measurement.station,measurement.tobs,measurement.date).\
    filter(station.station == 'USC00519523').all()
stn1

In [ ]:
# Create a new DataFrame that for the most active station
df_stn1 = df_measurement.loc[df_measurement["station"] == "USC00519523"]
df_stn1

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
max = df_stn1['tobs'].max()
min = df_stn1['tobs'].min()
avg = df_stn1['tobs'].mean()

print(f"The lowest temperature of the most active station id (USC00519523) is {min}.")
print(f"The highest temperature of the most active station id (USC00519523) is {max}.")
print(f"The average temperature of the most active station id (USC00519523) is {round(avg,2)}.")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
#Find out the latest date for the most active station
df_stn1 = df_stn1.sort_values(['date'], ascending=False)
df_stn1

In [ ]:
#Filter to generate the dataframe for plotting 
df_stn1 = df_stn1.loc[(df_stn1['date']>='2016-08-23'),['date','tobs']]
df_stn1

In [ ]:
# Determine which measure of central tendency is most appropriate to describe the Population
plt.hist(df_stn1['tobs'])
plt.xlabel('tobs')
plt.ylabel('Counts')
plt.show()
print(df_stn1['tobs'].mean())
print(df_stn1['tobs'].median())
print(df_stn1['tobs'].mode())

# Close session

In [ ]:
# Close Session
session.close()